<a href="https://colab.research.google.com/github/Lasal0425/Movie-Recommendation-System/blob/main/MovieRec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importiing the Dependencies

In [ ]:
import numpy as np
import pandas as pd
import difflib  #To find the closest match between the user provided string and the string that is in the dataset.
from sklearn.feature_extraction.text import TfidfVectorizer  #To convert the texual data into numerical values.
from sklearn.metrics.pairwise import cosine_similarity  #To find the similarity between the movies.

In [ ]:
!pip install flask flask-ngrok flask-cors

Data Collection and Pre-Processing

In [ ]:
#Loading the data from csv file to a pandas data set
movies_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Movie Recommendation System/movies.csv')

In [ ]:
#Printing the first 5 rows of the dataset
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [ ]:
#Number of rows and columns in the data frame
movies_data.shape

(4803, 24)

In [ ]:
#Selecting the relevent features for recommendation
selected_features = ['genres','keywords','tagline','cast','director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [ ]:
#Replace the null values with null string
for feature in selected_features:
  movies_data[feature] = movies_data[feature].fillna('')

In [ ]:
#Combining all the selected features
combined_features = movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['tagline']+' '+movies_data['cast']+' '+movies_data['director']

In [ ]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [ ]:
#Converting the text data to feature vectors

vectorizer = TfidfVectorizer()  #loading the TfidfVectrizer to vectorizer variable

In [ ]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [ ]:
print(feature_vectors)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 124266 stored elements and shape (4803, 17318)>
  Coords	Values
  (0, 201)	0.07860022416510505
  (0, 274)	0.09021200873707368
  (0, 5274)	0.11108562744414445
  (0, 13599)	0.1036413987316636
  (0, 5437)	0.1036413987316636
  (0, 3678)	0.21392179219912877
  (0, 3065)	0.22208377802661425
  (0, 5836)	0.1646750903586285
  (0, 14378)	0.33962752210959823
  (0, 16587)	0.12549432354918996
  (0, 3225)	0.24960162956997736
  (0, 14271)	0.21392179219912877
  (0, 4945)	0.24025852494110758
  (0, 15261)	0.07095833561276566
  (0, 16998)	0.1282126322850579
  (0, 11192)	0.09049319826481456
  (0, 11503)	0.27211310056983656
  (0, 13349)	0.15021264094167086
  (0, 17007)	0.23643326319898797
  (0, 17290)	0.20197912553916567
  (0, 13319)	0.2177470539412484
  (0, 14064)	0.20596090415084142
  (0, 16668)	0.19843263965100372
  (0, 14608)	0.15150672398763912
  (0, 8756)	0.22709015857011816
  :	:
  (4801, 403)	0.17727585190343229
  (4801, 4835)	0.247137650

Cosine Similarity


In [ ]:
#Getting the similarity score from cosine similarity
similarity = cosine_similarity(feature_vectors)

In [ ]:
print(similarity)

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


In [ ]:
print(similarity.shape)  #first value represent the movie's index and the second value represents the similarity score

(4803, 4803)


In [ ]:
#Getting the movie name from the user
movie_name = input(' Enter your favourite movie name : ')

 Enter your favourite movie name : superman


In [ ]:
#Creating a list with all the movies names given in the dataset
list_of_all_titles = movies_data['title'].tolist()  #tolist is used to create a list
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [ ]:
#Finding the close match for the movie name given by the user
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['Superman', 'Superman II', 'Superman III']


In [ ]:
close_match = find_close_match[0]  #Taking the first value
print(close_match)

Superman


In [ ]:
#Finding the index of the movie with the title
index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
print(index_of_the_movie)

813


In [ ]:
#Getting a list of similar movies
similarity_score = list(enumerate(similarity[index_of_the_movie])) #Enumerate is used to run a loop in a particular list
print(similarity_score)

[(0, np.float64(0.06605757327786128)), (1, np.float64(0.05494299286841541)), (2, np.float64(0.022474655275199852)), (3, np.float64(0.18229768118495995)), (4, np.float64(0.04826852550842614)), (5, np.float64(0.03115302093753189)), (6, np.float64(0.007816453213423849)), (7, np.float64(0.03428924698332947)), (8, np.float64(0.020070942434549842)), (9, np.float64(0.10380582477831074)), (10, np.float64(0.20133128757695568)), (11, np.float64(0.019513980745610133)), (12, np.float64(0.025827307032424)), (13, np.float64(0.016928223234223397)), (14, np.float64(0.31886105364073525)), (15, np.float64(0.018553049692010753)), (16, np.float64(0.03251349500357993)), (17, np.float64(0.024706817498456192)), (18, np.float64(0.029643169623505952)), (19, np.float64(0.04417369541584225)), (20, np.float64(0.02994794123287961)), (21, np.float64(0.011401180198063948)), (22, np.float64(0.03639562527544031)), (23, np.float64(0.025066323882186787)), (24, np.float64(0.03654732000312344)), (25, np.float64(0.0)), (26

In [ ]:
#Sorting the movies based on their similarity score
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)  #Arranging values from higher to lower
print(sorted_similar_movies)

[(813, np.float64(1.0)), (870, np.float64(0.4582408553739052)), (2433, np.float64(0.3864273245926594)), (14, np.float64(0.31886105364073525)), (1296, np.float64(0.2529397869699988)), (823, np.float64(0.21578853909502332)), (10, np.float64(0.20133128757695568)), (428, np.float64(0.18596641221431343)), (72, np.float64(0.1851290630861619)), (2793, np.float64(0.18272642077822945)), (3, np.float64(0.18229768118495995)), (1420, np.float64(0.1727129308212663)), (119, np.float64(0.17180650719081145)), (65, np.float64(0.16160582880593954)), (3337, np.float64(0.15158767314441426)), (4401, np.float64(0.14678890114385917)), (4267, np.float64(0.1452043635201541)), (1359, np.float64(0.1441412830396247)), (210, np.float64(0.14402535832262303)), (1238, np.float64(0.1306893940867623)), (1282, np.float64(0.12714887083401077)), (587, np.float64(0.12579274308188748)), (2492, np.float64(0.1253438322333649)), (164, np.float64(0.12497636481940003)), (1024, np.float64(0.1241984655709511)), (4432, np.float64(0

In [ ]:
#Print the name of similar movies based on the index
print('Movies suggested for you : \n')

i = 1

for movies in sorted_similar_movies:
  index = movies[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

Movies suggested for you : 

1 . Superman
2 . Superman II
3 . Superman IV: The Quest for Peace
4 . Man of Steel
5 . Superman III
6 . Crimson Tide
7 . Superman Returns
8 . Batman Returns
9 . Suicide Squad
10 . The Killer Inside Me
11 . The Dark Knight Rises
12 . Nanny McPhee and the Big Bang
13 . Batman Begins
14 . The Dark Knight
15 . The Godfather
16 . The Helix... Loaded
17 . Batman
18 . Batman
19 . Batman & Robin
20 . The Island of Dr. Moreau
21 . The Hunting Party
22 . The Abyss
23 . Steel
24 . Lethal Weapon 4
25 . Dick Tracy
26 . On the Waterfront
27 . 1941
28 . Star Trek IV: The Voyage Home
29 . Don Juan DeMarco


In [ ]:
#Getting the movie name from the user
movie_name = input(' Enter your favourite movie name : ')

#Creating a list with all the movies names given in the dataset
list_of_all_titles = movies_data['title'].tolist()  #tolist is used to create a list


#Finding the close match for the movie name given by the user
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)


close_match = find_close_match[0]  #Taking the first value


#Finding the index of the movie with the title
index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]


#Getting a list of similar movies
similarity_score = list(enumerate(similarity[index_of_the_movie])) #Enumerate is used to run a loop in a particular list


#Sorting the movies based on their similarity score
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)  #Arranging values from higher to lower


#Print the name of similar movies based on the index
print('Movies suggested for you : \n')

i = 1

for movies in sorted_similar_movies:
  index = movies[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# For Colab + ngrok
!pip install pyngrok --quiet
from pyngrok import ngrok

# Set your ngrok authtoken here. Replace "YOUR_NGROK_AUTHTOKEN" with your actual token.
# You can get your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken
ngrok.set_auth_token("30sCBSUstcgL0Ud3dXrHpQjNbks_2wXjhLU3X1yGcfdT3JMZX")


app = Flask(__name__)
CORS(app)

# Global variables
movies_data = None
similarity = None

def load_model():
    global movies_data, similarity

    print("Loading movie data and creating similarity matrix...")

    movies_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Movie Recommendation System/movies.csv')  # Update path if needed

    selected_features = ['genres','keywords','tagline','cast','director']
    for feature in selected_features:
        movies_data[feature] = movies_data[feature].fillna('')

    combined_features = movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['tagline']+' '+movies_data['cast']+' '+movies_data['director']

    vectorizer = TfidfVectorizer()
    feature_vectors = vectorizer.fit_transform(combined_features)

    similarity = cosine_similarity(feature_vectors)

    print(f"Similarity matrix shape: {similarity.shape}")
    print("Model loaded successfully!")

@app.route('/recommend', methods=['POST'])
def recommend_movies():
    try:
        data = request.get_json()
        movie_name = data.get('movie_name', '')

        if not movie_name:
            return jsonify({'error': 'Movie name is required'}), 400

        list_of_all_titles = movies_data['title'].tolist()
        find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

        if not find_close_match:
            return jsonify({'error': 'No similar movie found. Please try a different movie name.'}), 404

        close_match = find_close_match[0]

        if 'index' in movies_data.columns:
            index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
        else:
            index_of_the_movie = movies_data[movies_data.title == close_match].index[0]

        similarity_score = list(enumerate(similarity[index_of_the_movie]))
        sorted_similar_movies = sorted(similarity_score, key=lambda x: x[1], reverse=True)

        recommendations = []
        count = 0

        for movies in sorted_similar_movies:
            if count >= 30:
                break

            index = movies[0]

            if 'index' in movies_data.columns:
                title_from_index = movies_data[movies_data['index'] == index]['title'].values
            else:
                title_from_index = movies_data.iloc[index]['title']
                title_from_index = [title_from_index]

            if len(title_from_index) > 0:
                title = title_from_index[0]
                score = movies[1]
                if title != close_match:
                    count += 1
                    recommendations.append({
                        'rank': count,
                        'title': title,
                        'similarity_score': float(score)
                    })

        return jsonify({
            'input_movie': close_match,
            'recommendations': recommendations,
            'total_recommendations': len(recommendations)
        })

    except Exception as e:
        return jsonify({'error': f'An error occurred: {str(e)}'}), 500

@app.route('/health', methods=['GET'])
def health_check():
    return jsonify({
        'status': 'healthy',
        'message': 'Movie recommendation API is running',
        'total_movies': len(movies_data) if movies_data is not None else 0
    })

@app.route('/movies', methods=['GET'])
def get_all_movies():
    try:
        if movies_data is None:
            return jsonify({'error': 'Model not loaded'}), 500

        movies_list = movies_data['title'].tolist()
        return jsonify({
            'total_movies': len(movies_list),
            'sample_movies': movies_list[:20]
        })
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/search', methods=['GET'])
def search_movies():
    try:
        query = request.args.get('q', '')
        if not query:
            return jsonify({'error': 'Query parameter q is required'}), 400

        list_of_all_titles = movies_data['title'].tolist()
        matches = difflib.get_close_matches(query, list_of_all_titles, n=10, cutoff=0.3)

        return jsonify({
            'query': query,
            'matches': matches
        })
    except Exception as e:
        return jsonify({'error': str(e)}), 500

# ========= Colab-compatible run =============
print("="*60)
print("MOVIE RECOMMENDATION API")
print("="*60)

try:
    load_model()
except Exception as e:
    print(f"Error loading model: {e}")
    exit(1)

print("Starting Flask server...")
print("API endpoints:")
print("  /health")
print("  /movies")
print("  /search?q=batman")
print("  /recommend (POST)")
print("="*60)

# Set ngrok tunnel
port = 5000
public_url = ngrok.connect(port)
print(f" * ngrok tunnel: {public_url}")

# Run Flask app
app.run(port=port)

MOVIE RECOMMENDATION API
Loading movie data and creating similarity matrix...
Similarity matrix shape: (4803, 4803)
Model loaded successfully!
Starting Flask server...
API endpoints:
  /health
  /movies
  /search?q=batman
  /recommend (POST)
 * ngrok tunnel: NgrokTunnel: "https://649c545702b6.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [05/Aug/2025 15:23:03] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [05/Aug/2025 15:23:03] "GET /favicon.ico HTTP/1.1" 404 -
